In [2]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os
import cv2
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
num_classes=7
img_rows,img_cols=48,48
batch_size=32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_data_dir='/content/drive/My Drive/Colab Notebooks/train'
validation_data_dir='/content/drive/My Drive/Colab Notebooks/test'

train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    color_mode='grayscale',
                    target_size=(img_rows,img_cols),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            color_mode='grayscale',
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical',
                            shuffle=True)

Found 1173 images belonging to 7 classes.
Found 293 images belonging to 7 classes.


In [18]:
def addModel(model,features,shape,kernelInit,activationFunction,poolSize,dropoutRatio):
  model.add(Conv2D(features,shape,padding='same',kernel_initializer=kernelInit,input_shape=(img_rows,img_cols,1)))
  model.add(Activation(activationFunction))
  model.add(BatchNormalization())
  model.add(Conv2D(features,shape,padding='same',kernel_initializer=kernelInit,input_shape=(img_rows,img_cols,1)))
  model.add(Activation(activationFunction))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=poolSize))
  model.add(Dropout(dropoutRatio))
  return model

model = Sequential()
kernelInit = 'he_normal'
activationFunction = 'elu'
addModel(model,16,(3,3),kernelInit,activationFunction,(2,2),0.2)
addModel(model,32,(3,3),kernelInit,activationFunction,(2,2),0.2)
addModel(model,64,(3,3),kernelInit,activationFunction,(2,2),0.2)
addModel(model,128,(3,3),kernelInit,activationFunction,(2,2),0.2)
addModel(model,256,(3,3),kernelInit,activationFunction,(2,2),0.2)

model.add(Flatten())
model.add(Dense(64,kernel_initializer=kernelInit))
model.add(Activation(activationFunction))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64,kernel_initializer=kernelInit))
model.add(Activation(activationFunction))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(num_classes,kernel_initializer=kernelInit))
model.add(Activation('softmax'))

In [19]:
print(model.summary())


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 48, 48, 16)        160       
_________________________________________________________________
activation_41 (Activation)   (None, 48, 48, 16)        0         
_________________________________________________________________
batch_normalization_36 (Batc (None, 48, 48, 16)        64        
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 48, 48, 16)        2320      
_________________________________________________________________
activation_42 (Activation)   (None, 48, 48, 16)        0         
_________________________________________________________________
batch_normalization_37 (Batc (None, 48, 48, 16)        64        
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 24, 24, 16)       

In [20]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]


In [21]:
model.compile(loss='categorical_crossentropy',
 optimizer = Adam(lr=0.001),
 metrics=['accuracy'])                                            

nb_train_samples = 1173
nb_validation_samples = 293
epochs=25

history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

Epoch 1/25
36/36 [==============================] - ETA: 0s - loss: 3.2146 - accuracy: 0.1350
Epoch 00001: val_loss improved from inf to 2.08737, saving model to EmotionDetectionModel.h5
36/36 [==============================] - 4s 114ms/step - loss: 3.2146 - accuracy: 0.1350 - val_loss: 2.0874 - val_accuracy: 0.0382
Epoch 2/25
36/36 [==============================] - ETA: 0s - loss: 2.7725 - accuracy: 0.1849
Epoch 00002: val_loss improved from 2.08737 to 1.81422, saving model to EmotionDetectionModel.h5
36/36 [==============================] - 4s 99ms/step - loss: 2.7725 - accuracy: 0.1849 - val_loss: 1.8142 - val_accuracy: 0.1875
Epoch 3/25
36/36 [==============================] - ETA: 0s - loss: 2.5041 - accuracy: 0.2147
Epoch 00003: val_loss did not improve from 1.81422
36/36 [==============================] - 3s 96ms/step - loss: 2.5041 - accuracy: 0.2147 - val_loss: 1.9298 - val_accuracy: 0.1736
Epoch 4/25
36/36 [==============================] - ETA: 0s - loss: 2.2070 - accuracy: